In [0]:
import numpy as np

import tensorflow as tf

import numpy as np
import time, math
from tqdm import tqdm_notebook as tqdm


import tensorflow.contrib.eager as tfe
tf.enable_eager_execution()

In [2]:
import os
os.getcwd()
from google.colab import drive
import os
drive.mount('/content/gdrive')

root_path = 'gdrive/My Drive/Colab Notebooks/eva_research_team4/src/' #change dir to your project folder

os.chdir('/content/' + root_path)
os.getcwd()

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


'/content/gdrive/My Drive/Colab Notebooks/eva_research_team4/src'

In [3]:
!git branch

  augmentation
  densenext
  development
  master
  tfrecords_fix
* zeedense


In [4]:
!git pull

Already up to date.


In [5]:
#from all_models import DavidNet

import run_util


from importlib import reload
reload(run_util)

from run_util import Run

import data_pipeline

reload(data_pipeline)

from zeedensenet import ZeeDenseNet

Using TensorFlow backend.



32 conv0.27260135918100314 (3, 3) (1, 1) same (1, 1) None glorot_uniform
32 conv0.27260135918100314 (3, 3) (1, 1) same (1, 1) None glorot_uniform
32 conv0.27260135918100314 (3, 3) (1, 1) same (1, 1) None glorot_uniform


In [5]:
data_pipeline.get_data(dataset_name = "CIFAR10", tfrecords_flag=True)

saving to tf records
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded cifar-10-python.tar.gz 170498071 bytes.
Generating ../data/train/train.tfrecords


Generating ../data/eval/eval.tfrecords
getting tf records complete


######

In [0]:
#model = ZeeDenseNet(dimensions_dict= {"dimensions_to_sample":(8,8)}, layers_filters={0:16, 1:32})

obj = Run()

In [0]:
train_mean = np.array([125.30691805, 122.95039414, 113.86538318])

train_std= np.array([62.99321928, 62.08870764, 66.70489964])

normalize = lambda x: ((x - train_mean) / train_std)

def data_aug_train(x, y):
    
    #x = tf.image.per_image_standardization(x)
    
    x = normalize(x)
        
    #x = tf.image.random_flip_left_right(x)
    
    paddings = [(4, 4), (4, 4), (0, 0)]
    
    x = tf.pad(x, paddings, "REFLECT")
    
    x = tf.random_crop(x, [32, 32, 3])
    
    return (x, y)
  
def data_aug_test(x, y):
    
    #x = tf.image.per_image_standardization(x)
    
    x = normalize(x)
        
    #x = tf.image.random_flip_left_right(x)
    
    #paddings = [(4, 4), (4, 4), (0, 0)]
    
    #x = tf.pad(x, paddings, "REFLECT")
    
    #x = tf.random_crop(x, [32, 32, 3])
    
    return (x, y)

# Train

In [0]:
BATCH_SIZE = 512 #@param {type:"integer"}
MOMENTUM = 0.95 #@param {type:"number"}

MIN_MOMENTUM = 0.8 #@param {type:"number"}
LEARNING_RATE = 0.4 #@param {type:"number"}
WEIGHT_DECAY = 5e-4 #@param {type:"number"}
EPOCHS = 30 #@param {type:"integer"}


MIN_LEARNING_RATE = 0.000001 #@param {type:"number"}

END_LR_SMOOTHING_PERC = 0.15 #@param {type:"number"}

COMMENTS = "Zeedensenet multisoftmax, gap on channels" #@param {type:"string"}


params_tune = {
    
  "epochs":EPOCHS, 
 
  "batch_size" : BATCH_SIZE,

  "max_lr": LEARNING_RATE,

  "min_lr":MIN_LEARNING_RATE,

  "end_anneal_pc":END_LR_SMOOTHING_PERC,

  "max_mom":MOMENTUM,
 
  "min_mom":MIN_MOMENTUM,
 
  "wd":WEIGHT_DECAY,
  
  "skip_testing_epochs":0,
    
  "batches_per_epoch":50000//BATCH_SIZE,
    
  "comments":COMMENTS
}

In [0]:
loaded_tfrecs = data_pipeline.load_tfrecords(params_tune["batch_size"])

train_dataset = loaded_tfrecs["train"]

eval_dataset = loaded_tfrecs["eval"]

In [0]:
def tst_data_supplier(epoch_num):
    
    batch_size = params_tune["batch_size"]
    
    global eval_dataset
  
    len_test = 10000

    test_set = eval_dataset.map(data_aug_test).batch(batch_size).prefetch(1)
    
    return (test_set, len_test)

def trn_data_supplier(epoch_num):
    
    batch_size = params_tune["batch_size"]
  
    global train_dataset
    
    len_train = 50000

    train_set = train_dataset.map(data_aug_train).batch(batch_size).prefetch(1)
    
    return (train_set, len_train)

In [14]:
#import cProfile
model2 = ZeeDenseNet(f_filter=16,  
                     dimensions_dict= {"dimensions_to_sample":(8,8)}, 
                     gap_mode="channel_axis",
                     layers_filters={0:8, 1:16},
                    multisoft_list = [0, 1, 2])
x = obj.run(params_tune, trn_data_supplier, tst_data_supplier, model = model2)

16 0.8031016948496695conv (3, 3) (1, 1) same (1, 1) None glorot_uniform
32 0.7991130454577566conv (3, 3) (1, 1) same (1, 1) None glorot_uniform
32 0.4917460292034307conv (3, 3) (1, 1) same (1, 1) None glorot_uniform
32 0.5403579815635616conv (3, 3) (1, 1) same (1, 1) None glorot_uniform
64 0.4777684201780009conv (3, 3) (1, 1) same (1, 1) None glorot_uniform
64 0.6103264373579538conv (3, 3) (1, 1) same (1, 1) None glorot_uniform
64 0.8500714806728025conv (3, 3) (1, 1) same (1, 1) None glorot_uniform
128 0.6611443193700529conv (3, 3) (1, 1) same (1, 1) None glorot_uniform
128 0.5313123336669981conv (3, 3) (1, 1) same (1, 1) None glorot_uniform
128 0.48477475624467947conv (3, 3) (1, 1) same (1, 1) None glorot_uniform
8 0.044375522254821886conv (3, 3) (1, 1) same (2, 2) None glorot_uniform
8 0.3259446518313355conv (3, 3) (1, 1) same (2, 2) None glorot_uniform
8 0.9073061754711111conv (3, 3) (1, 1) same (2, 2) None glorot_uniform
8 0.07713148702073569conv (3, 3) (1, 1) same (2, 2) None glor

epoch: 1 lr: 0.06666666666666667 train loss: 3.288 train acc:0.261 val loss: 2.665 val acc: 0.417 time:183.967
epoch: 2 lr: 0.13333333333333333 train loss: 2.356 train acc:0.501 val loss: 2.162 val acc: 0.523 time:361.764
epoch: 3 lr: 0.2 train loss: 1.845 train acc:0.628 val loss: 2.211 val acc: 0.569 time:539.822
epoch: 4 lr: 0.26666666666666666 train loss: 1.596 train acc:0.691 val loss: 1.560 val acc: 0.705 time:718.380
epoch: 5 lr: 0.3333333333333333 train loss: 1.464 train acc:0.730 val loss: 1.518 val acc: 0.700 time:896.464
epoch: 6 lr: 0.4 train loss: 1.336 train acc:0.766 val loss: 1.313 val acc: 0.747 time:1074.915
epoch: 7 lr: 0.3810526315789474 train loss: 1.223 train acc:0.792 val loss: 1.337 val acc: 0.756 time:1253.057
epoch: 8 lr: 0.36210526315789476 train loss: 1.156 train acc:0.809 val loss: 1.244 val acc: 0.792 time:1431.484
epoch: 9 lr: 0.3431578947368421 train loss: 1.088 train acc:0.825 val loss: 1.414 val acc: 0.780 time:1609.085
epoch: 10 lr: 0.3242105263157895

In [15]:
obj.model.summary()

Model: "zee_dense_net"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv_bn_rl_3 (ConvBnRl)      multiple                  496       
_________________________________________________________________
res_blk (ResBlk)             multiple                  23424     
_________________________________________________________________
res_blk_1 (ResBlk)           multiple                  92928     
_________________________________________________________________
res_blk_2 (ResBlk)           multiple                  370176    
_________________________________________________________________
zee_conv_blk (ZeeConvBlk)    multiple                  10560     
_________________________________________________________________
global_max_pooling2d (Global multiple                  0         
_________________________________________________________________
dense_3 (Dense)              multiple                

ValueError: ignored

In [0]:
obj.show_missclassified_images(num_images=10, tst_data_supplier=tst_data_supplier)

In [0]:
obj.plot_confusion_matrix(tst_data_supplier=tst_data_supplier)

### Grid Search

### LR Finder

Creating a separate data supplier for lr finder

In [0]:
def tst_data_supplier_lr(epoch_num):
    
    batch_size = params_tune["batch_size"]
    
    global eval_dataset
  
    len_test = 100

    test_set = eval_dataset.take(100).map(data_aug_test).batch(batch_size).prefetch(1)
    
    return (test_set, len_test)

def trn_data_supplier_lr(epoch_num):
    
    batch_size = params_tune["batch_size"]
  
    global train_dataset
    
    len_train = 1500

    train_set = train_dataset.take(len_train).map(data_aug_train).batch(batch_size).prefetch(1)
    
    return (train_set, len_train)

In [0]:
obj.lr_finder(model_fn, lr_list=[  0.004, 0.007, 0.01, 0.05, 0.09, 0.1, 0.4, 0.7, 0.9, 1], tst_data_supplier=tst_data_supplier_lr,
             trn_data_supplier=trn_data_supplier_lr)

In [0]:
model1 = ZeeDenseNet(dimensions_dict= {"dimensions_to_sample":(8,8)}, layers_filters={0:16})

model2 = ZeeDenseNet(dimensions_dict= {"dimensions_to_sample":(8,8)}, layers_filters={0:16, 1:32})

model3 = ZeeDenseNet(dimensions_dict= {"dimensions_to_sample":(8,8)}, layers_filters={0:16, 1:32, 2:64})

model4 = ZeeDenseNet(f_filter=32, dimensions_dict= {"dimensions_to_sample":(8,8)}, layers_filters={0:16})

model5 = ZeeDenseNet(f_filter=16, dimensions_dict= {"dimensions_to_sample":(8,8)}, layers_filters={0:16, 1:32, 2:64})

model6 = ZeeDenseNet(f_filter=32, dimensions_dict= {"dimensions_to_sample":(8,8)}, layers_filters={0:16, 1:32})

model6 = ZeeDenseNet(f_filter=32, dimensions_dict= {"dimensions_to_sample":(8,8)}, layers_filters={0:16, 1:32, 2:64})

params_tune_grid = {
    
    "model": [model1, model2, model3, model4, model5, model6],
    
  "epochs":[30] ,
 
  "batch_size" : [256],

  "max_lr": [0.1, 0.4, 1, 10, 0.01],

  "min_lr":[MIN_LEARNING_RATE],

  "end_anneal_pc":[END_LR_SMOOTHING_PERC],

  "max_mom":[MOMENTUM],
 
  "min_mom":[MIN_MOMENTUM],
 
  "wd":[WEIGHT_DECAY],
  
  "skip_testing_epochs":[0],
    
  "batches_per_epoch":[50000//BATCH_SIZE],
    
   "comments":["zeedense grid search"]
}


obj.grid_search(params_tune_grid, trn_data_supplier, tst_data_supplier)

##### Viewing log

In [0]:
import pandas as pd

pd.read_csv("../data/run_logger.csv").tail()

#### Numpy implementation

x_train, y_train, x_test, y_test = data_pipeline.load_saved_numpy_data(
    train_path=["../data/train/train_x.npy","../data/train/train_y.npy"],
    test_path=["../data/test/test_x.npy", "../data/test/test_y.npy"])

def tst_data_supplier(epoch_num):
    
    batch_size = params_tune["batch_size"]
    
    global eval_dataset
  
    len_test = x_test[0].shape[0]

    test_set = tf.data.Dataset.from_tensor_slices((x_test[:30,:,:,:],
                                                   y_test[:30])).batch(batch_size).prefetch(1)
    
    return (test_set, len_test)

def trn_data_supplier(epoch_num):
    
    batch_size = params_tune["batch_size"]
  
    global y_train
    
    global x_train
    
    len_test = x_train[0].shape[0]

    test_set = tf.data.Dataset.from_tensor_slices((x_train[:50,:,:,:], 
                                        y_train[:50])).batch(batch_size).prefetch(1)
    
    return (test_set, len_test)

def tst_data_supplier_lr(epoch, test_num_batches=2):
 
    batch_size = params_tune["batch_size"]
    
    global x_test
  
    global y_test
  
    sample_idx = np.random.choice(y_test.size, test_num_batches * batch_size, replace=False)

    x_test_lr = x_test[sample_idx, :, :, :]

    y_test_lr = y_test[sample_idx]
    
    len_test = x_test_lr[0].shape[0]

    test_set = tf.data.Dataset.from_tensor_slices((x_test_lr,
                                                   y_test_lr)).batch(batch_size).prefetch(1)

    return (test_set, len_test)

def trn_data_supplier_lr(epoch, train_num_batches=2):
 
    batch_size = params_tune["batch_size"]
    
    global x_train
  
    global y_train
  
    sample_idx = np.random.choice(y_train.size, train_num_batches * batch_size, replace=False)

    x_train_lr = x_train[sample_idx, :, :, :]

    y_train_lr = y_train[sample_idx]
    
    len_train = x_train_lr[0].shape[0]

    train_set = tf.data.Dataset.from_tensor_slices((x_train_lr,
                                                   y_train_lr)).batch(batch_size).prefetch(1)

    return (train_set, len_train)